# 加载数据

In [1]:
import json
import polars as pl

In [1]:
import json
with open("/opt/tiger/LLM/face/data/face_pos.jsonl") as f:
    data = []
    for line in f.readlines():
        data.append(json.loads(line))
print(len(data))
print(data[0])

1000
{'object_id': 'tiktok_live@7364652435727567662_1714718428000', 'url_list': ['http://lf16-fcdn-timeshift.tiktokcdn-us.com/obj/fcdn-tiktok-tx/71465123841433646761/push-rtmp-f5-tt01.fcdn.us.tiktokv.com/stage/stream-3573837222899614640/20240503064015749.jpg', 'http://p-lab-va.bytedance.net/obj/post-meta-va/71465123841433646761/push-rtmp-f5-tt01.fcdn.us.tiktokv.com/stage/stream-3573837222899614640/tiktok-v2/20240503064017888.jpg', 'http://p-lab-va.bytedance.net/obj/post-meta-va/71465123841433646761/push-rtmp-f5-tt01.fcdn.us.tiktokv.com/stage/stream-3573837222899614640/tiktok-v2/20240503064019939.jpg', 'http://p-lab-va.bytedance.net/obj/post-meta-va/71465123841433646761/push-rtmp-f5-tt01.fcdn.us.tiktokv.com/stage/stream-3573837222899614640/tiktok-v2/20240503064021931.jpg', 'http://p-lab-va.bytedance.net/obj/post-meta-va/71465123841433646761/push-rtmp-f5-tt01.fcdn.us.tiktokv.com/stage/stream-3573837222899614640/tiktok-v2/20240503064023873.jpg', 'http://p-lab-va.bytedance.net/obj/post-met

In [5]:
import polars as pl
data = pl.read_csv("/opt/tiger/labeled_pos_2000/part-00000-81a3d2d2-ed66-4051-96ca-b6bf309f0ccf-c000.csv")
print(data.columns)
print(data.head(10))
print(len(data))

['object_id', 'image_urls', 'room_id']
shape: (10, 3)
┌─────────────────────────────────┬─────────────────────────────────┬─────────────────────┐
│ object_id                       ┆ image_urls                      ┆ room_id             │
│ ---                             ┆ ---                             ┆ ---                 │
│ str                             ┆ str                             ┆ i64                 │
╞═════════════════════════════════╪═════════════════════════════════╪═════════════════════╡
│ tiktok_live@735665948258953088… ┆ [{"offline_uri":"http://webcas… ┆ 7356659482589530886 │
│ tiktok_live@735665948258953088… ┆ [{"offline_uri":"http://webcas… ┆ 7356659482589530886 │
│ tiktok_live@735776848083165466… ┆ [{"offline_uri":"http://webcas… ┆ 7357768480831654662 │
│ tiktok_live@735776848083165466… ┆ [{"offline_uri":"http://webcas… ┆ 7357768480831654662 │
│ tiktok_live@735934514826785873… ┆ [{"offline_uri":"http://tosv.t… ┆ 7359345148267858731 │
│ tiktok_live@735934514826

In [20]:
import polars as pl
import json

def trans2zeus_feature_pos(row):
    url_list = []
    image_urls = json.loads(row[1])
    for img_dict in image_urls:
        img_url = img_dict['offline_uri']
        img_url = img_url.replace('tosv.byted.org', 'p-lab-va.bytedance.net').replace('tosv.tiktokd.org', 'p-lab-va.bytedance.net').replace('post-meta-tx', 'post-meta-va').replace('fcdn-live-triage-audit-tx', 'fcdn-live-triage-audit-va')
        # 新规则
        img_url = img_url.replace('http://webcast-review-tos-va.byteintl.net/obj/post-meta-va', 'http://p-lab-va.bytedance.net/obj/post-meta-va').replace('http://webcast-review-tos.byteintl.net', 'http://p-lab-va.bytedance.net').replace('post-meta-euttp', 'post-meta-va').replace('fcdn-tiktok-euttp', 'fcdn-tiktok').replace('fcdn-live-triage-audit-euttp', 'fcdn-live-triage-audit-va')
        img_url = img_url.replace('tosv-ttp2.tiktokd.org', 'p-lab-va.bytedance.net')
        url_list.append(img_url)

    return (row[0], json.dumps(url_list), row[2])

# 应用转换函数到Polars DataFrame的每一行
feature_list = data.map_rows(trans2zeus_feature_pos)
feature_list.head()

column_0,column_1,column_2
str,str,i64
"""tiktok_live@735665948258953088…","""[""http://p-lab-va.bytedance.ne…",7356659482589530886
"""tiktok_live@735665948258953088…","""[""http://p-lab-va.bytedance.ne…",7356659482589530886
"""tiktok_live@735776848083165466…","""[""http://p-lab-va.bytedance.ne…",7357768480831654662
"""tiktok_live@735776848083165466…","""[""http://p-lab-va.bytedance.ne…",7357768480831654662
"""tiktok_live@735934514826785873…","""[""http://p-lab-va.bytedance.ne…",7359345148267858731


In [21]:
rename = {"column_0":"object_id", "column_1":'image_urls', "column_2":"room_id"}
feature_list = feature_list.rename(rename)
feature_list.columns

['object_id', 'image_urls', 'room_id']

In [22]:
feature_list = feature_list.to_dicts()
processed_df = []
for i in range(0, len(feature_list), 2):
    tmp = {}
    tmp["room_id"] = feature_list[i]["room_id"]
    tmp["object1"] = feature_list[i]["object_id"]
    tmp["urls1"] = feature_list[i]["image_urls"]
    tmp["object2"] = feature_list[i+1]["object_id"]
    tmp["urls2"] = feature_list[i+1]["image_urls"]
    processed_df.append(tmp)
print(len(processed_df))

1000


In [23]:
print(processed_df[0])

{'room_id': 7356659482589530886, 'object1': 'tiktok_live@7356659482589530886_1714780789000', 'urls1': '["http://p-lab-va.bytedance.net/obj/post-meta-va/7654592258592283018/push-rtmp-l10-va01.tiktokcdn.com/stage/stream-2997251568003187449/tiktok-v2/20240503235931548.jpg", "http://p-lab-va.bytedance.net/obj/post-meta-va/7654592258592283018/push-rtmp-l10-va01.tiktokcdn.com/stage/stream-2997251568003187449/tiktok-v2/20240503235933578.jpg", "http://p-lab-va.bytedance.net/obj/post-meta-va/7654592258592283018/push-rtmp-l10-va01.tiktokcdn.com/stage/stream-2997251568003187449/tiktok-v2/20240503235935557.jpg", "http://p-lab-va.bytedance.net/obj/post-meta-va/7654592258592283018/push-rtmp-l10-va01.tiktokcdn.com/stage/stream-2997251568003187449/tiktok-v2/20240503235937558.jpg", "http://p-lab-va.bytedance.net/obj/post-meta-va/7654592258592283018/push-rtmp-l10-va01.tiktokcdn.com/stage/stream-2997251568003187449/tiktok-v2/20240503235939562.jpg", "http://p-lab-va.bytedance.net/obj/post-meta-va/76545922

In [2]:
with open("/mnt/bn/data-tns-live-llm/leon/experiments/llm/face/processed_pair_face.json", "r") as f:
    data = json.loads(f.read())

In [4]:
from genericpath import exists
import requests
import uuid
import os
root = "/mnt/bn/data-tns-live-llm/leon/experiments/llm/face/second_stage_train_imgs"
if not os.path.exists(f'{root}'): os.mkdir(f'{root}')
def get_img_roomid(data):
    if not os.path.exists(f'{root}/{data["room_id"]}/'): 
        os.mkdir(f'{root}/{data["room_id"]}/')
        os.mkdir(f'{root}/{data["room_id"]}/{data["object1"]}')
        os.mkdir(f'{root}/{data["room_id"]}/{data["object2"]}')
    else: return None
    urls1, urls2 = json.loads(data["urls1"]), json.loads(data["urls2"])
    for url in urls1:
        try:
            response = requests.get(url)
            # 检查请求是否成功
            if response.status_code == 200:
                # 使用uuid生成唯一的文件名
                file_name = str(uuid.uuid4()) + '.jpg'
                # 构建文件保存路径
                file_path = os.path.join(f'{root}/{data["room_id"]}/{data["object1"]}', file_name)
                # 将响应内容写入文件
                with open(file_path, 'wb') as file:
                    file.write(response.content)
            else:
                print('图片下载失败，状态码:', response.status_code)
        except Exception as e:
            print(f'图片下载失败，错误：{e}')

    for url in urls2:
        try:
            response = requests.get(url)
            # 检查请求是否成功
            if response.status_code == 200:
                # 使用uuid生成唯一的文件名
                file_name = str(uuid.uuid4()) + '.jpg'
                # 构建文件保存路径
                file_path = os.path.join(f'{root}/{data["room_id"]}/{data["object2"]}', file_name)
                # 将响应内容写入文件
                with open(file_path, 'wb') as file:
                    file.write(response.content)
            else:
                print('图片下载失败，状态码:', response.status_code)
        except Exception as e:
            print(f'图片下载失败，错误：{e}')

In [24]:
# 将转换后的数据写入文件
save_df = json.dumps(processed_df, ensure_ascii=False, indent=4)
with open("/mnt/bn/data-tns-live-llm/leon/experiments/llm/face/processed_pair_face.json", "w") as f:
    f.write(save_df)

In [2]:
import os
import json
base = "/opt/tiger/url_processed_json/"
paths = os.listdir(base)
data = []
for path in paths:
    with open(base+path) as f:
        for line in f.readlines():
            data.append(json.loads(line))
print(len(data))

10000


In [5]:
print(data[0])

{'object_id': 'tiktok_live@7362733341105998597_1714780867000', 'room_id': '7362733341105998597', 'url_list': ['http://lp10-va01.tiktokcdn.com/stage/stream-2997346472147288825/1714780559000.jpg', 'http://lp10-va01.tiktokcdn.com/stage/stream-2997346472147288825/1714780563000.jpg', 'http://lp10-va01.tiktokcdn.com/stage/stream-2997346472147288825/1714780567000.jpg', 'http://webcast-review-tos-va.byteintl.net/obj/thunder-audit-backup-us/5807046414045719637/push-rtmp-l10-va01.tiktokcdn.com/stage/stream-2997346472147288825/20240503235613947.jpeg', 'http://lp10-va01.tiktokcdn.com/stage/stream-2997346472147288825/1714780685000.jpg', 'http://lp10-va01.tiktokcdn.com/stage/stream-2997346472147288825/1714780689000.jpg', 'http://webcast-review-tos-va.byteintl.net/obj/thunder-audit-backup-us/5807046414045719637/push-rtmp-l10-va01.tiktokcdn.com/stage/stream-2997346472147288825/20240503235814539.jpeg', 'http://webcast-review-tos-va.byteintl.net/obj/thunder-audit-backup-us/5807046414045719637/push-rtmp-

In [4]:
import requests
import uuid
import os

def get_img(url):
    try:
        response = requests.get(url)
        # 检查请求是否成功
        if response.status_code == 200:
            # 使用uuid生成唯一的文件名
            file_name = str(uuid.uuid4()) + '.jpg'
            # 构建文件保存路径
            file_path = os.path.join('/mnt/bn/data-tns-live-llm/leon/experiments/llm/face/test_imgs/', file_name)
            
            # 将响应内容写入文件
            with open(file_path, 'wb') as file:
                file.write(response.content)
            return file_path  # 返回文件保存的路径

        else:
            print('图片下载失败，状态码:', response.status_code)
            return None

    except Exception as e:
        print(f'图片下载失败，错误：{e}')
        return None

In [10]:
from genericpath import exists
import requests
import uuid
import os

def get_test_img(data):
    if not os.path.exists(f'/mnt/bn/data-tns-live-llm/leon/experiments/llm/face/test_imgs/{data["room_id"]}/'): os.mkdir(f'/mnt/bn/data-tns-live-llm/leon/experiments/llm/face/test_imgs/{data["room_id"]}/')
    for url in data["url_list"]:
        try:
            response = requests.get(url)
            # 检查请求是否成功
            if response.status_code == 200:
                # 使用uuid生成唯一的文件名
                file_name = str(uuid.uuid4()) + '.jpg'
                # 构建文件保存路径
                file_path = os.path.join(f'/mnt/bn/data-tns-live-llm/leon/experiments/llm/face/test_imgs/{data["room_id"]}/', file_name)
                # 将响应内容写入文件
                with open(file_path, 'wb') as file:
                    file.write(response.content)
            else:
                print('图片下载失败，状态码:', response.status_code)
        except Exception as e:
            print(f'图片下载失败，错误：{e}')

In [ ]:
imgs = []
for d in data:
    for url in d["url_list"]:
        imgs.append(get_img(url))
print(len(d))

In [4]:
import multiprocessing
from functools import partial

# 从数据中提取所有的URL
all_urls = []
for d in data:
    all_urls.extend(d["url_list"])

In [6]:
import random

# 确保列表中有足够的 URL
if len(all_urls) >= 200000:
    # 随机选择 10 万个 URL
    selected_urls = random.sample(all_urls, 200000)
else:
    print("URL 数量不足 10 万！")

# 现在 selected_urls 包含了随机选择的 10 万个 URL

In [ ]:
# 使用多进程池来下载图像
def download_images(urls):
    with multiprocessing.Pool() as pool:
        pool.map(get_img, urls)

# 调用函数开始多进程下载
download_images(selected_urls)

# get test imgs

In [5]:
import multiprocessing
# 使用多进程池来下载图像
def download_images(data):
    with multiprocessing.Pool() as pool:
        pool.map(get_img_roomid, data)

# 调用函数开始多进程下载
download_images(data)

In [6]:
tmp = os.listdir("/mnt/bn/data-tns-live-llm/leon/experiments/llm/face/second_stage_train_imgs")
print(len(tmp))

1000


In [12]:
import cv2
import numpy as np

img_array = []
for i, img in enumerate(imgs):
    # 将字节类型的数据解码成图像数组
    img_np = np.frombuffer(img, dtype=np.uint8)  # 首先将字节数据转换为NumPy数组
    img_decoded = cv2.imdecode(img_np, cv2.IMREAD_COLOR)  # 然后解码图像
    if img_decoded is not None:  # 检查解码是否成功
        img_array.append(img_decoded)  # 如果解码成功，添加到列表中
    else:
        print(f"解码索引为{i}的图像失败")

# 检查是否所有图像都解码成功
if len(img_array) == len(imgs):
    print(f"成功解码了{len(img_array)}张图像。")
    for i, img in enumerate(img_array):
        # 将图像数组保存为JPG格式的文件
        cv2.imwrite(f'/opt/tiger/LLM/face/data/downloaded_imgs/{i}.jpg', img)
else:
    print(f"总共有{len(imgs)}张图像，但只成功解码了{len(img_array)}张。")

print(img_array[0].shape)  # 打印第一张图像的形状，如果它存在

成功解码了10张图像。
(1280, 720, 3)


# 提取人脸

In [7]:
import os
import euler
import argparse
import numpy as np
import sys
import cv2
sys.path.append("/opt/tiger/LLM/face")
from fermion_core_thrift import *
from base_thrift import *

def get_color_map_list(num_classes):
    """
    Args:
        num_classes (int): number of class
    Returns:
        color_map (list): RGB color list
    """
    color_map = num_classes * [0, 0, 0]
    for i in range(0, num_classes):
        j = 0
        lab = i
        while lab:
            color_map[i * 3] |= (((lab >> 0) & 1) << (7 - j))
            color_map[i * 3 + 1] |= (((lab >> 1) & 1) << (7 - j))
            color_map[i * 3 + 2] |= (((lab >> 2) & 1) << (7 - j))
            j += 1
            lab >>= 3
    color_map = [color_map[i:i + 3] for i in range(0, len(color_map), 3)]
    return color_map

def preprocess(np_boxes: list[float]):
    np_boxes = np.array(np_boxes)
    np_boxes = np_boxes.reshape(-1, 6).tolist()
    preprocessed_boxes = []
    for box in np_boxes:
        # 分离类别ID和边界框坐标以及分数
        clsid, x1, y1, x2, y2, score = int(box[5]), box[0], box[1], box[2], box[3], box[4]
        # 将边界框坐标打包成一个列表
        bbox = [x1, y1, x2, y2]
        # 将信息添加到新列表中
        preprocessed_boxes.append([clsid, score, bbox])
    return preprocessed_boxes
    
def draw_box(im, np_boxes, labels, threshold=0.5):
    """
    Args:
        im (PIL.Image.Image): PIL image
        np_boxes (np.ndarray): shape:[N,6], N: number of box,
                               matix element:[class, score, x_min, y_min, x_max, y_max]
        labels (list): labels:['class1', ..., 'classn']
        threshold (float): threshold of box
    Returns:
        im (PIL.Image.Image): visualized image
    """
    np_boxes = preprocess(np_boxes)
    draw_thickness = min(im.size) // 320
    draw = ImageDraw.Draw(im)
    clsid2color = {}
    color_list = get_color_map_list(len(labels))
    # print("np_boxes", np_boxes)
    np_boxes = [
        box for box in np_boxes
        if box[1] > threshold and box[0] > -1
    ]
    # np_boxes = np_boxes[expect_boxes, :]

    for dt in np_boxes:
        clsid, bbox, score = int(dt[0]), dt[2], dt[1]
        if clsid not in clsid2color:
            clsid2color[clsid] = color_list[clsid]
        color = tuple(clsid2color[clsid])
        color = (22,222,22)

        if len(bbox) == 4:
            xmin, ymin, xmax, ymax = bbox
            # print('class_id:{:d}, confidence:{:.4f}, left_top:[{:.2f},{:.2f}],'
            #       'right_bottom:[{:.2f},{:.2f}]'.format(
            #           int(clsid), score, xmin, ymin, xmax, ymax))
            # draw bbox
            draw.line(
                [(xmin, ymin), (xmin, ymax), (xmax, ymax), (xmax, ymin),
                 (xmin, ymin)],
                width=4,
                fill=color)
        elif len(bbox) == 8:
            x1, y1, x2, y2, x3, y3, x4, y4 = bbox
            draw.line(
                [(x1, y1), (x2, y2), (x3, y3), (x4, y4), (x1, y1)],
                width=2,
                fill=color)
            xmin = min(x1, x2, x3, x4)
            ymin = min(y1, y2, y3, y4)

        # draw label
        text = "{} {:.4f}".format(labels[clsid], score)
        tw, th = draw.textsize(text)
        draw.rectangle(
            [(xmin + 1, ymin - th), (xmin + tw + 1, ymin)], fill=color)
        draw.text((xmin + 1, ymin - th), text, fill=(255, 255, 255))
    return im

def InferImages(file_content, client):
    req = InferRequest()
    t = Tensor()
    t.dtype = DataType.STRING
    t.shape = []
    t.str_data = [file_content]
    req.input = [TensorSet(tensors={"image": t})]
    resp = client.Infer(req)
    # print(resp)
    return resp.output[0]  # batch=1, so index 0 is the final result

client = euler.Client(FermionCore, 'sd://tns.cv.facedet_offline.service.sg1', timeout=5000)

/usr/local/lib/python3.9/dist-packages/euler/hook.py:26: EulerDeprecatedWarning: Importing a thrift file without user's explicit intention had caused various troubles and is deprecated by now. Implicit installation of the import hook will be removed in Euler 2.0.
Either state your intention by calling `euler.install_thrift_import_hook()`, or use `thriftpy2.load()` to manually load thrift files.
  warnings.warn(msg, EulerDeprecatedWarning)
/usr/local/lib/python3.9/dist-packages/euler/client.py:137: EulerDeprecatedWarning: Don't specify idc in PSM with `sd://p.s.m.service.${IDC}` format, otherwise it will cause abnormal behaviors in MS or Mesh. The right format is `sd://p.s.m?idc=${IDC}`.
  warnings.warn(msg, errors.EulerDeprecatedWarning)
/usr/local/lib/python3.9/dist-packages/euler/client.py:143: EulerDeprecatedWarning: cluster not specified, by some history reasons, all clusters will be used without mesh mode, but only default cluster will be used in mesh mode. So we highly recommend 

## 统计有效下载图片数量

In [9]:
root = "/mnt/bn/data-tns-live-llm/leon/experiments/llm/face/second_stage_train_imgs/"
cnt = 0
for d in data:
    cnt += len(os.listdir(f'{root}/{d["room_id"]}/{d["object1"]}'))
    cnt += len(os.listdir(f'{root}/{d["room_id"]}/{d["object2"]}'))
print(cnt)

19630


In [16]:
with open("/mnt/bn/data-tns-live-llm/leon/experiments/llm/face/downloaded_imgs/00352d75-f266-4625-a742-b02562b6b8ad.jpg", "rb") as f:
    img = f.read()
    output_tensor = InferImages(img, client)
    print(output_tensor)

TensorSet(tensors={'prob': Tensor(dtype=232, shape=[6], int8_data=[], int16_data=[], int32_data=[], int64_data=[], float_data=[146.1750030517578, 347.8781433105469, 341.25, 637.6218872070312, 0.92626953125, 0.0], str_data=[], data=b'', extra={})}, extra={})


In [ ]:
# img_dir = "/opt/tiger/LLM/face/data/downloaded_imgs"

# for i, img in enumerate(imgs):
#     output_tensor = InferImages(img, client)
#     np_bbox = np.array(output_tensor.tensors['prob'].float_data)
#     bbox_shape = output_tensor.tensors['prob'].shape
#     np_bbox = np_bbox.reshape(bbox_shape)
#     print("np_bbox",np_bbox)
#     if sum(np_bbox)==0: 
#         print("没有人脸")
#         continue
#     if len(np_bbox) > 6:
#         print("多张人脸")
#         continue

#     # visualization 可视化对其效果
#     from PIL import Image, ImageDraw
#     img_np = np.frombuffer(img, dtype=np.uint8)  # 首先将字节数据转换为NumPy数组
#     img = cv2.imdecode(img_np, cv2.IMREAD_COLOR)  # 然后解码图像
#     # 从 np_bbox 中提取边界框的坐标
#     x1, y1, x2, y2 = np_bbox[:4].astype(int)
#     # 根据边界框坐标裁剪图像
#     cropped_img = img[y1:y2, x1:x2]
#     cv2.imwrite(f'/opt/tiger/LLM/face/data/downloaded_imgs/{i}.jpg', cropped_img)

# 读取 img

## 一阶段无监督

In [ ]:
import multiprocessing
import uuid
import cv2
import numpy as np
from multiprocessing import Pool

imgs_dic = "/mnt/bn/data-tns-live-llm/leon/experiments/llm/face/test_imgs/"
target_path = "/mnt/bn/data-tns-live-llm/leon/experiments/llm/face/cropped_test_imgs"
# 定义处理单个图像的函数
def process_image(paths):
    imgs_path = os.listdir(imgs_dic+paths)
    os.mkdir(os.path.join(target_path,paths))
    for path in imgs_path:
        try:
            with open(os.path.join(imgs_dic,paths,path), "rb") as f:
                img = f.read()
            if img is None:
                raise ValueError(f"无法读取图像: {path}")
            
            output_tensor = InferImages(img, client)
            np_bbox = np.array(output_tensor.tensors['prob'].float_data)
            bbox_shape = output_tensor.tensors['prob'].shape
            np_bbox = np_bbox.reshape(bbox_shape)
            if len(np_bbox) != 6:
                continue
            if sum(np_bbox) == 0:
                continue
            x1, y1, x2, y2 = np_bbox[:4].astype(int)

            from PIL import Image, ImageDraw
            img_np = np.frombuffer(img, dtype=np.uint8)  # 首先将字节数据转换为NumPy数组
            img = cv2.imdecode(img_np, cv2.IMREAD_COLOR)  # 然后解码图像
            cropped_img = img[y1:y2, x1:x2]
            
            # 使用uuid生成唯一的文件名
            # file_name = str(uuid.uuid4()) + '.jpg'
            # 保存裁剪后的图像
            cv2.imwrite(os.path.join(target_path,paths,path), cropped_img)
        except Exception as e:
            print(f"处理图像出错: {path}, 错误: {e}")

# 使用多进程池来并行处理图像
def parallel_process_images(paths):
    with Pool() as pool:
        # 使用pool.map来并行处理每个图像
        results = pool.map(process_image, paths)
    return results

## 二阶段有监督

In [15]:
import multiprocessing
import uuid
import cv2
import numpy as np
from multiprocessing import Pool

root = "/mnt/bn/data-tns-live-llm/leon/experiments/llm/face/second_stage_train_imgs"
target_path = "/mnt/bn/data-tns-live-llm/leon/experiments/llm/face/cropped_second_stage_imgs"
if not os.path.exists(target_path): os.mkdir(target_path)
# 定义处理单个图像的函数
def process_image(data):
    path1 = os.path.join(root, str(data["room_id"]), data["object1"])
    path2 = os.path.join(root, str(data["room_id"]), data["object2"])
    target_path1 = os.path.join(target_path, str(data["room_id"]), data["object1"])
    target_path2 = os.path.join(target_path, str(data["room_id"]), data["object2"])
    imgs1 = os.listdir(path1)
    imgs2 = os.listdir(path2)
    os.makedirs(target_path1, exist_ok=True)
    os.makedirs(target_path2, exist_ok=True)
    for path in imgs1:
        try:
            with open(os.path.join(path1, path), "rb") as f:
                img = f.read()
            if img is None:
                raise ValueError(f"无法读取图像: {path}")
            output_tensor = InferImages(img, client)
            np_bbox = np.array(output_tensor.tensors['prob'].float_data)
            bbox_shape = output_tensor.tensors['prob'].shape
            np_bbox = np_bbox.reshape(bbox_shape)
            if len(np_bbox) != 6:
                continue
            if sum(np_bbox) == 0:
                continue
            x1, y1, x2, y2 = np_bbox[:4].astype(int)
            from PIL import Image, ImageDraw
            img_np = np.frombuffer(img, dtype=np.uint8)  # 首先将字节数据转换为NumPy数组
            img = cv2.imdecode(img_np, cv2.IMREAD_COLOR)  # 然后解码图像
            cropped_img = img[y1:y2, x1:x2]
            # 保存裁剪后的图像
            cv2.imwrite(os.path.join(target_path1, path), cropped_img)
        except Exception as e:
            print(f"处理图像出错: {path}, 错误: {e}")

    for path in imgs2:
        try:
            with open(os.path.join(path1, path), "rb") as f:
                img = f.read()
            if img is None:
                raise ValueError(f"无法读取图像: {path}")
            output_tensor = InferImages(img, client)
            np_bbox = np.array(output_tensor.tensors['prob'].float_data)
            bbox_shape = output_tensor.tensors['prob'].shape
            np_bbox = np_bbox.reshape(bbox_shape)
            if len(np_bbox) != 6:
                continue
            if sum(np_bbox) == 0:
                continue
            x1, y1, x2, y2 = np_bbox[:4].astype(int)
            from PIL import Image, ImageDraw
            img_np = np.frombuffer(img, dtype=np.uint8)  # 首先将字节数据转换为NumPy数组
            img = cv2.imdecode(img_np, cv2.IMREAD_COLOR)  # 然后解码图像
            cropped_img = img[y1:y2, x1:x2]
            # 保存裁剪后的图像
            cv2.imwrite(os.path.join(target_path2, path), cropped_img)
        except Exception as e:
            print(f"处理图像出错: {path}, 错误: {e}")

# 使用多进程池来并行处理图像
def parallel_process_images(paths):
    with Pool(20) as pool:
        # 使用pool.map来并行处理每个图像
        results = pool.map(process_image, paths)
    return results

In [16]:
# 假设 imgs_dic 是一个包含图像文件路径的字典
# paths = os.listdir(imgs_dic)
# 调用函数开始多进程处理
results = parallel_process_images(data)

In [10]:
tmp = os.listdir("/mnt/bn/data-tns-live-llm/leon/experiments/llm/face/croped_face/")
print(len(tmp))

114530


In [11]:
print(tmp[:10])

['00005acf-9c26-4cca-b0d5-46d1e83572a5.jpg', '0000ee9b-bf37-4773-9b0d-ca6045d729e0.jpg', '0001058a-e0a2-4a89-8587-9915b1dd8839.jpg', '0001507b-3d0c-4a8c-92d0-01d9ece1e9f2.jpg', '00017397-148a-4b14-8787-221f744b28d4.jpg', '00028d17-86ef-4389-afff-b93ab349ba7d.jpg', '00032153-1e9b-4c94-b0b7-ee3dbf139bd1.jpg', '000348af-a7cc-43d5-8685-a600dfc4f7af.jpg', '0004550a-c8a7-4485-86b1-89028f4c7b80.jpg', '0004ff12-0d54-4c10-ac95-ff10a342b7f3.jpg']


In [12]:
import os
import shutil
from sklearn.model_selection import train_test_split

# 图片存放的原始文件夹路径
original_data_path = "/mnt/bn/data-tns-live-llm/leon/experiments/llm/face/croped_face/"

# 训练集和验证集的目标文件夹路径
train_data_path = "/mnt/bn/data-tns-live-llm/leon/experiments/llm/face/croped_face/train/face"
val_data_path = "/mnt/bn/data-tns-live-llm/leon/experiments/llm/face/croped_face/val/face"

# 确保目标文件夹存在
os.makedirs(train_data_path, exist_ok=True)
os.makedirs(val_data_path, exist_ok=True)

# 获取所有图片的路径
all_image_paths = [os.path.join(original_data_path, f) for f in os.listdir(original_data_path) if os.path.isfile(os.path.join(original_data_path, f))]

# 将图片路径分为训练集和验证集，比例为 9:1
train_paths, val_paths = train_test_split(all_image_paths, test_size=0.1, random_state=42)

# 移动图片到对应的训练集和验证集文件夹
for path in train_paths:
    shutil.copy(path, train_data_path)

for path in val_paths:
    shutil.copy(path, val_data_path)

print(f"图片分为训练集和验证集完成，训练集图片数量：{len(train_paths)}, 验证集图片数量：{len(val_paths)}")

图片分为训练集和验证集完成，训练集图片数量：103077, 验证集图片数量：11453


In [ ]:
if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument("--file")
    parser.add_argument("--dir")
    args = parser.parse_args(["--dir","/opt/tiger/LLM/face/data/"])
    client = euler.Client(FermionCore, 'sd://tns.cv.facedet_offline.service.sg1', timeout=5000)

    img_file = args.file 
    img_dir = args.dir 

    if img_file:

        with open(img_file, mode='rb') as f:
            file_content = f.read()

        output_tensor = InferImages(file_content, client)
        np_bbox = np.array(output_tensor.tensors['bbox'].float_data)
        bbox_shape = output_tensor.tensors['bbox'].shape
        np_bbox = np_bbox.reshape(bbox_shape)
        print(np_bbox)
        
        # visualization 可视化对其效果
        from PIL import Image, ImageDraw
        im = Image.open(img_file).convert('RGB')
        drawed_im = draw_box(im, np_bbox, labels=['face',], threshold=0.4)
        im.save('det_result.jpg', quality=95)

    if img_dir:
        file_list = os.listdir(img_dir)
        for img_file in file_list:
            resfile_name = '/opt/tiger/LLM/face/data/result/' + img_file
            img_file = os.path.join(img_dir, img_file)
            with open(img_file, mode='rb') as f:
                file_content = f.read()

            output_tensor = InferImages(file_content, client)
            np_bbox = np.array(output_tensor.tensors['prob'].float_data)
            bbox_shape = output_tensor.tensors['prob'].shape
            np_bbox = np_bbox.reshape(bbox_shape)
            print(np_bbox)
            
            # visualization 可视化对其效果
            from PIL import Image, ImageDraw
            im = Image.open(img_file).convert('RGB')
            drawed_im = draw_box(im, np_bbox, labels=['pip',], threshold=0.6)
            im.save(resfile_name, quality=95)

In [39]:
import torch
checkpoint = torch.load("/opt/tiger/moco/moco_v1_200ep_pretrain.pth.tar")

In [ ]:
print(checkpoint.keys())
print(checkpoint["arch"])
def clean_state_dict(state_dict, prefix='module.'):
    """
    去掉state_dict中的指定前缀。
    """
    keys = sorted([key for key in state_dict if key.startswith(prefix)])
    for key in keys:
        state_dict[key.replace(prefix, '')] = state_dict[key]
        del state_dict[key]
    return state_dict

state_dict = clean_state_dict(checkpoint["state_dict"],"module.encoder_q.")
print(state_dict.keys())

In [2]:
import polars as pl
data = pl.read_csv("/opt/tiger/labeled_pos_2000/part-00000-81a3d2d2-ed66-4051-96ca-b6bf309f0ccf-c000.csv")
print(data.columns)
print(data.head())

['object_id', 'image_urls', 'room_id']
shape: (5, 3)
┌─────────────────────────────────┬─────────────────────────────────┬─────────────────────┐
│ object_id                       ┆ image_urls                      ┆ room_id             │
│ ---                             ┆ ---                             ┆ ---                 │
│ str                             ┆ str                             ┆ i64                 │
╞═════════════════════════════════╪═════════════════════════════════╪═════════════════════╡
│ tiktok_live@735665948258953088… ┆ [{"offline_uri":"http://webcas… ┆ 7356659482589530886 │
│ tiktok_live@735665948258953088… ┆ [{"offline_uri":"http://webcas… ┆ 7356659482589530886 │
│ tiktok_live@735776848083165466… ┆ [{"offline_uri":"http://webcas… ┆ 7357768480831654662 │
│ tiktok_live@735776848083165466… ┆ [{"offline_uri":"http://webcas… ┆ 7357768480831654662 │
│ tiktok_live@735934514826785873… ┆ [{"offline_uri":"http://tosv.t… ┆ 7359345148267858731 │
└──────────────────────────